### [`36 Pandas & Numpy：策略与回测系统`](https://learn.lianglianglee.com/%e4%b8%93%e6%a0%8f/Python%e6%a0%b8%e5%bf%83%e6%8a%80%e6%9c%af%e4%b8%8e%e5%ae%9e%e6%88%98/36%20Pandas%20&%20Numpy%ef%bc%9a%e7%ad%96%e7%95%a5%e4%b8%8e%e5%9b%9e%e6%b5%8b%e7%b3%bb%e7%bb%9f.md)


#### OHLCV 数据


In [10]:
import os
import numpy as np
import pandas as pd
import yfinance as yf

from utils import (
    assert_msg,
    read_file,
    crossover,
    SMA
)

from exchange_api import ExchangeAPI
from strategy_api import Strategy
from backtesting import Backtest

In [11]:
# df_asset = read_file("BTCUSD_GEMINI.csv")[
#     ['Open', 'High', 'Low', 'Close', 'Volume']].sort_index(inplace=False)


df_asset = yf.download(
    "AAPL",
    start='2010-01-01',
    # end='2023-09-28',
)[['Open', 'High', 'Low', 'Close', 'Volume']].sort_index(inplace=False)

[*********************100%%**********************]  1 of 1 completed


In [12]:
df_asset.head(3)

,Open,High,Low,Close,Volume
Date,,,,,
2010-01-04,7.622500,7.660714,7.585000,7.643214,493729600
2010-01-05,7.664286,7.699643,7.616071,7.656429,601904800
2010-01-06,7.656429,7.686786,7.526786,7.534643,552160000


In [13]:
df_asset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3476 entries, 2010-01-04 to 2023-10-24
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    3476 non-null   float64
 1   High    3476 non-null   float64
 2   Low     3476 non-null   float64
 3   Close   3476 non-null   float64
 4   Volume  3476 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 162.9 KB


In [14]:
df_asset.index[0]

Timestamp('2010-01-04 00:00:00')

#### 回测框架

1. 读取 OHLC 数据；
2. 对 OHLC 进行指标运算；
3. 策略根据指标向量决定买卖；
4. 发给模拟的”交易所“进行交易；
5. 最后，统计结果。

对此，使用之前学到的面向对象思维方式，我们可以大致抽取三个类：

- 交易所类（ ExchangeAPI）：负责维护账户的资金和仓位，以及进行模拟的买卖；
- 策略类（Strategy）：负责根据市场信息生成指标，根据指标决定买卖；
- 回测类框架（Backtest）：包含一个策略类和一个交易所类，负责迭代地对每个数据点调用策略执行。

回到正题，至此，我们就确定了 Backtest 的输入和输出。它的输入是：

- OHLC 数据
- 初始资金
- 手续费率
- 交易所类
- 策略类

输出则是：最后剩余市值。


这段代码有点长，但是核心其实就两部分。

- 初始化函数（init）：传入必要参数，对 OHLC 数据进行简单清洗、排序和验证。我们从不同地方下载的数据，可能格式不一样；而排序的方式也可能是从前往后。所以，这里我们把数据统一设置为按照时间从之前往现在的排序。
- 执行函数（run）：这是回测框架的主要循环部分，核心是更新市场还有更新策略的时间。迭代完成所有的历史数据后，它会计算收益并返回。

你应该注意到了，此时，我们还没有定义策略和交易所 API 的结构。不过，通过回测的执行函数，我们可以确定这两个类的接口形式。

策略类（Strategy）的接口形式为：

- 初始化函数 init()，根据历史数据进行指标（Indicator）计算。
- 步进函数 next()，根据当前时间和指标，决定买卖操作，并发给交易所类执行。

交易所类（ExchangeAPI）的接口形式为：

- 步进函数 next()，根据当前时间，更新最新的价格；
- 买入操作 buy()，买入资产；
- 卖出操作 sell()，卖出资产。


#### 交易策略


In [18]:
class SmaCross(Strategy):
    # 小窗口SMA的窗口大小，用于计算SMA快线
    fast = 5

    # 大窗口SMA的窗口大小，用于计算SMA慢线
    slow = 15

    def init(self):
        # 计算历史上每个时刻的快线和慢线
        self.sma1 = self.I(SMA, self.data.Close, self.fast)
        self.sma2 = self.I(SMA, self.data.Close, self.slow)

    def next(self, tick):
        # 如果此时快线刚好越过慢线，买入全部
        if crossover(self.sma1[:tick], self.sma2[:tick]):
            self.buy()

        # 如果是慢线刚好越过快线，卖出全部
        elif crossover(self.sma2[:tick], self.sma1[:tick]):
            self.sell()

        # 否则，这个时刻不执行任何操作。
        else:
            pass

In [19]:

backtestObj = Backtest(
    df_asset,
    SmaCross,
    ExchangeAPI,
    10000.0,
    0.00
)
ret = backtestObj.run()

_market_value - 76167.07879664477
_market_value - 76167.07879664477


In [20]:
ret

初始市值    10000.000000
结束市值    76167.078797
收益      66167.078797
dtype: float64

In [44]:
backtestObj = Backtest(
    df_asset,
    SmaCross,
    ExchangeAPI,
    10000.0,
    0.00
)

In [45]:
_broker = ExchangeAPI(
    data=df_asset,
    cash=10000.0,
    commission=0.00
)

In [46]:
_strategy = SmaCross(
    broker=_broker,
    data=df_asset
)

In [49]:
_strategy._indicators

[]

In [50]:
_strategy.init()

In [51]:
_strategy._indicators

[array([      nan,       nan,       nan, ..., 11418.37 , 11427.523,
        11442.361]),
 array([       nan,        nan,        nan, ..., 11333.7895, 11350.099 ,
        11361.247 ])]